In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
# Use OpenRouter API key instead of OpenAI
openrouter_api_key = os.environ["OPENROUTER_API_KEY"]
openrouter_model = os.environ.get("OPENROUTER_MODEL", "openai/gpt-3.5-turbo")

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [3]:
loader = TextLoader("data\Rocket.txt")
loaded_document = loader.load()

In [4]:
loaded_document

[Document(metadata={'source': 'data\\Rocket.txt'}, page_content="Here are some detailed examples of how Python is used in rocket technology:\n1. Simulation and Modeling\nTrajectory Simulation: Python is used to simulate the trajectory of rockets, taking into account factors like gravity, thrust, and atmospheric conditions.\nDynamics Simulation: Python is used to simulate the dynamics of rocket systems, such as the motion of rocket components, thermal analysis, and structural analysis.\nSome popular Python libraries used for simulation and modeling include:\nNumPy: For numerical computations and data analysis.\nSciPy: For scientific computing and signal processing.\nPandas: For data manipulation and analysis.\nMatplotlib: For data visualization.\n2. Data Analysis\nTelemetry Data Analysis: Python is used to analyze telemetry data from rockets, including data from sensors, cameras, and other instruments.\nPerformance Analysis: Python is used to analyze the performance of rocket systems, i

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks_of_text = text_splitter.split_documents(loaded_document)

In [6]:
len(chunks_of_text)

1

In [7]:
# Use HuggingFace embeddings instead of OpenAI
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vector_db = FAISS.from_documents(chunks_of_text, embeddings)

C:\Windows\Temp\ipykernel_5292\920948181.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [8]:
vector_db

In [9]:
retriever = vector_db.as_retriever()

In [27]:
response = retriever.invoke("what is python in Rocket?")

In [28]:
response

[Document(id='1174f9b8-d68e-4f17-97dc-aef77a675df0', metadata={'source': 'data\\Rocket.txt'}, page_content="Here are some detailed examples of how Python is used in rocket technology:\n1. Simulation and Modeling\nTrajectory Simulation: Python is used to simulate the trajectory of rockets, taking into account factors like gravity, thrust, and atmospheric conditions.\nDynamics Simulation: Python is used to simulate the dynamics of rocket systems, such as the motion of rocket components, thermal analysis, and structural analysis.\nSome popular Python libraries used for simulation and modeling include:\nNumPy: For numerical computations and data analysis.\nSciPy: For scientific computing and signal processing.\nPandas: For data manipulation and analysis.\nMatplotlib: For data visualization.\n2. Data Analysis\nTelemetry Data Analysis: Python is used to analyze telemetry data from rockets, including data from sensors, cameras, and other instruments.\nPerformance Analysis: Python is used to a

In [12]:
len(response)

1

In [13]:
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

In [29]:
response = retriever.invoke("what is rocket?")

In [30]:
response

[Document(id='1174f9b8-d68e-4f17-97dc-aef77a675df0', metadata={'source': 'data\\Rocket.txt'}, page_content="Here are some detailed examples of how Python is used in rocket technology:\n1. Simulation and Modeling\nTrajectory Simulation: Python is used to simulate the trajectory of rockets, taking into account factors like gravity, thrust, and atmospheric conditions.\nDynamics Simulation: Python is used to simulate the dynamics of rocket systems, such as the motion of rocket components, thermal analysis, and structural analysis.\nSome popular Python libraries used for simulation and modeling include:\nNumPy: For numerical computations and data analysis.\nSciPy: For scientific computing and signal processing.\nPandas: For data manipulation and analysis.\nMatplotlib: For data visualization.\n2. Data Analysis\nTelemetry Data Analysis: Python is used to analyze telemetry data from rockets, including data from sensors, cameras, and other instruments.\nPerformance Analysis: Python is used to a

## Simple use with LCEL

In [16]:
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [17]:
template = """Answer the question based only on the following context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# Use OpenRouter instead of OpenAI
model = ChatOpenAI(
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=openrouter_api_key,
    model_name=openrouter_model,
)

C:\Windows\Temp\ipykernel_5292\11782002.py:11: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(


In [18]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [25]:
response = chain.invoke("what is python in Rocket?")

In [26]:
response

"Python is a programming language that is widely used in rocket technology for tasks such as simulation and modeling, data analysis, control systems, testing and automation, and scientific computing. It is used for a variety of purposes including trajectory simulation, telemetry data analysis, autopilot systems, testing automation, astrodynamics calculations, and more. Python's versatility, ease of use, and large community make it a popular choice for many applications in the space industry."

In [24]:
type(response)

str